# DATA_AutoAddImages.ipynb
Created by: `Panawun P.` <br>
Created on: `2025-09`<br>
Last editted: `2025-10-22`<br>
<br>
Manage Canopy Cover Images (Spherical Densiometer) and add to PCA Data spreadsheet, add transect photos to datasheet
- Currently need photos to be renamed first for the code to work

---

## Settings and Init


In [ ]:
RAW_DATA_SHEET_ID = '[GOOGLE DRIVE FILE ID]'

COLLECTION_SHEET_NAME = 'CollectionData'
CANOPY_SHEET_NAME = 'CanopyData'

CC_IMG_FOLDER_ID = '[GOOGLE DRIVE FOLDER ID]'
TRANSECT_IMG_FOLDER_ID = '[GOOGLE DRIVE FOLDER ID]'
COUNTED_CC_IMG_FOLDER_ID = '[GOOGLE DRIVE FOLDER ID]'


In [ ]:
# OAuth for gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()

import requests
import gspread

from googleapiclient.discovery import build
drive_service = build('drive', 'v3', credentials=creds)
# drive_scope = 'https://www.googleapis.com/auth/drive'

# gc = gspread.api_key(SHEET_API_KEY) # Access through API key
gc = gspread.authorize(creds) # Authorize with OAuth

## Functions

In [ ]:
def get_img_list(folder_id):
    query = f"'{folder_id}' in parents and trashed =false"
    file_list = drive_service.files().list(q=query,
                                        fields='nextPageToken, files(id, name, mimeType)').execute()
    img_list = file_list.get('files', [])
    img_list[0]['mimeType']

    img_list = [file_info for file_info in img_list if 'image/' in file_info['mimeType']]

    return img_list

In [ ]:
# keyword = ['start', 'end', 'trail', 'away']
def add_img_to_drive(keyword, img_list, row_i, row_info, worksheet):
    # Convert number to letter
    int_to_letter = lambda x:"" if x==0 else int_to_letter((x-1)//26)+chr((x-1)%26+ord("A"))

    if keyword == 'transect':
        img = [img_info for img_info in img_list][0] # Only use the first photo from transect images
        cell_col = row_info['transect_img_col']
    else:
        img = [img_info for img_info in img_list if keyword in img_info['name']][0]
        cell_col = row_info[f'CC{keyword}_col']

    drive_img_link = f'https://drive.google.com/file/d/{img['id']}/view?usp=sharing'

    # Insert each image into the row
    cell_location = f'{int_to_letter(cell_col+1)}{row_i+1}'
    link_formula = f'=HYPERLINK("{drive_img_link}","{img['name']}")'
    worksheet.update(cell_location, [[link_formula]], value_input_option='USER_ENTERED')

In [ ]:
# Raw Data Sheet
def update_sheet(data_key):
    print("Reading raw data sheet...")
    if data_key == 'raw_data':
        sheet_name = COLLECTION_SHEET_NAME
        sheet_url = f'https://docs.google.com/spreadsheets/d/{RAW_DATA_SHEET_ID}/'
        CC_imglist = get_img_list(CC_IMG_FOLDER_ID)
        transect_imglist = get_img_list(TRANSECT_IMG_FOLDER_ID)
    elif data_key == 'data':
        sheet_name = CANOPY_SHEET_NAME
        # sheet_url = f'https://docs.google.com/spreadsheets/d/{DATA_SHEET_ID}/'
        sheet_url = f'https://docs.google.com/spreadsheets/d/{RAW_DATA_SHEET_ID}/'
        CC_imglist = get_img_list(COUNTED_CC_IMG_FOLDER_ID)

    spreadsheet = gc.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_values()

    # Processing Data
    print("Processing data...")
    for row_i, row in enumerate(data):
        if row_i == 0: # Header row
            row_info = {'transect_id_col': row.index('TRANSECT ID'),
                        'CCstart_col': row.index('CC Start'),
                        'CCend_col': row.index('CC End'),
                        'CCtrail_col': row.index('CC Trail'),
                        'CCaway_col': row.index('CC Away'),
                        }

            if data_key == 'raw_data': row_info['transect_img_col'] = row.index('TRANSECT PHOTO')

        else:
            transect_id = row[row_info['transect_id_col']]
            print(transect_id)

            # Get images with ID corresponding to transect id
            CC_transect_img = [img_info for img_info in CC_imglist if transect_id in img_info['name']]

            if len(CC_transect_img) != 4:
                print("Err: Image missing.")
            else:
                add_img_to_drive('start', CC_transect_img, row_i, row_info, worksheet)
                add_img_to_drive('end', CC_transect_img, row_i, row_info, worksheet)
                add_img_to_drive('trail', CC_transect_img, row_i, row_info, worksheet)
                add_img_to_drive('away', CC_transect_img, row_i, row_info, worksheet)

            if data_key == 'raw_data':
                transect_img = [img_info for img_info in transect_imglist if transect_id in img_info['name']]
                if len(transect_img) != 0:
                    add_img_to_drive('transect', transect_img, row_i, row_info, worksheet)

## Main Block

In [ ]:
update_sheet('raw_data')
update_sheet('data')

Reading raw data sheet...
Processing data...
BP-R-02


/tmp/ipython-input-1886584482.py:18: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_location, [[link_formula]], value_input_option='USER_ENTERED')


BP-M-04
BP-R-03
BP-R-04
BP-M-02
BP-R-05
BP-R-06
BP-M-01
BP-M-05
BP-R-01
BP-M-03
BP-R-07
BP-M-06
BP-R-08
BP-M-07
Reading raw data sheet...
Processing data...
BP-R-02
BP-M-04
BP-R-03
BP-R-04
BP-M-02
BP-R-05
BP-R-06
BP-M-01
BP-M-05
BP-R-01
BP-M-03
BP-R-07
BP-M-06
BP-R-08
BP-M-07
